# Amazon

### T5

In [1]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost', "T5_amazon_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 31.1 MB/s eta 0:00:00
Mounted at /content/drive
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:26:19,225] A new study created in memory with name: no-name-1d08045b-fe99-4cd9-83af-e005b708ecd8


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:26:21,350] Trial 0 finished with value: 0.27741160685302274 and parameters: {'n_estimators': 800, 'learning_rate': 0.15954794274998432, 'max_depth': 5, 'gamma': 2.190434455989029e-08, 'reg_alpha': 7.46292099514062e-06, 'reg_lambda': 0.00013632303347546802, 'colsample_bytree': 0.8799098875864708, 'subsample': 0.8779456713561624}. Best is trial 0 with value: 0.27741160685302274.
[I 2025-10-12 06:26:23,310] Trial 1 finished with value: 0.28460813655971046 and parameters: {'n_estimators': 100, 'learning_rate': 0.09215029194857259, 'max_depth': 4, 'gamma': 1.1162305728661233e-06, 'reg_alpha': 0.015606366732766457, 'reg_lambda': 1.6166378345951018e-06, 'colsample_bytree': 0.8677933631648572, 'subsample': 0.6156280986666873}. Best is trial 1 with value: 0.28460813655971046.
[I 2025-10-12 06:26:25,652] Trial 2 finished with value: 0.2577330463036506 and parameters: {'n_estimators': 2000, 'learning_rate': 0.21487036893701703, 'max_depth': 7, 'gamma': 0.00031460306714961036, 're


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1700
       learning_rate: 0.0028193891783707867
           max_depth: 10
               gamma: 1.0823621294804414e-08
           reg_alpha: 2.3374398895496878e-05
          reg_lambda: 0.00044856354247210453
    colsample_bytree: 0.6491194122275146
           subsample: 0.9408962894385736

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline       0.283   0.7488    0.1697
Optuna Tuned   0.336   0.7842    0.0807

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost/T5_amazon_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### DistilBERT

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join(os.path.dirname(config.CSV_FILE_PATH), "DistilBERT_amazon_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:40:36,349] A new study created in memory with name: no-name-97fdfbca-74a5-4000-b153-77a78121ea15


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:40:39,839] Trial 0 finished with value: 0.2849185846138217 and parameters: {'n_estimators': 1400, 'learning_rate': 0.0301011161444221, 'max_depth': 7, 'gamma': 1.4306464683472199e-06, 'reg_alpha': 4.148713483589062e-06, 'reg_lambda': 0.0003024728412028708, 'colsample_bytree': 0.7194917595061661, 'subsample': 0.6472194177685486}. Best is trial 0 with value: 0.2849185846138217.
[I 2025-10-12 06:40:45,740] Trial 1 finished with value: 0.2867406839876208 and parameters: {'n_estimators': 600, 'learning_rate': 0.01747055346551136, 'max_depth': 10, 'gamma': 8.248758592533632e-08, 'reg_alpha': 1.3560442628431847e-06, 'reg_lambda': 0.09647699918077537, 'colsample_bytree': 0.9254528058859959, 'subsample': 0.6475280704894005}. Best is trial 1 with value: 0.2867406839876208.
[I 2025-10-12 06:40:49,442] Trial 2 finished with value: 0.29141359528920585 and parameters: {'n_estimators': 1400, 'learning_rate': 0.013255581777238647, 'max_depth': 4, 'gamma': 0.0018184185395730414, 'reg_a


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 300
       learning_rate: 0.01780366479022081
           max_depth: 5
               gamma: 0.006372455113591292
           reg_alpha: 1.1958872517092027e-05
          reg_lambda: 8.087564152335576
    colsample_bytree: 0.9151868685398672
           subsample: 0.610732307429619

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2746   0.7460    0.1719
Optuna Tuned  0.3300   0.7812    0.1313

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/DistilBERT_amazon_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [3]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost', "RoBERTa_amazon_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:46:11,427] A new study created in memory with name: no-name-38326d56-1876-40ba-9be4-a6257af54aa7


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:46:19,621] Trial 0 finished with value: 0.2892859047267671 and parameters: {'n_estimators': 1400, 'learning_rate': 0.0032795491380736715, 'max_depth': 5, 'gamma': 3.0918270850544893e-06, 'reg_alpha': 0.0008939323598130105, 'reg_lambda': 2.07729569619303e-06, 'colsample_bytree': 0.9637445091458682, 'subsample': 0.6755638256578145}. Best is trial 0 with value: 0.2892859047267671.
[I 2025-10-12 06:46:22,697] Trial 1 finished with value: 0.2593058472655543 and parameters: {'n_estimators': 1200, 'learning_rate': 0.09060148419581375, 'max_depth': 9, 'gamma': 0.0005570374086963475, 'reg_alpha': 0.10884503566601951, 'reg_lambda': 1.2180237230510142e-05, 'colsample_bytree': 0.9410165647210925, 'subsample': 0.6999371801675294}. Best is trial 0 with value: 0.2892859047267671.
[I 2025-10-12 06:46:56,587] Trial 2 finished with value: 0.2862181369158812 and parameters: {'n_estimators': 1200, 'learning_rate': 0.001869539585173886, 'max_depth': 12, 'gamma': 0.8736847148686803, 'reg_al


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 900
       learning_rate: 0.006883369413836256
           max_depth: 6
               gamma: 0.9599545101320001
           reg_alpha: 2.4214280151990296e-05
          reg_lambda: 9.88549883078152e-07
    colsample_bytree: 0.6574039434745261
           subsample: 0.7639146065811827

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2836   0.7419     0.168
Optuna Tuned  0.3294   0.7732     0.131

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost/RoBERTa_amazon_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [4]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost', "SentenceBERT_amazon_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:51:50,481] A new study created in memory with name: no-name-0eaef7f1-eaf6-4bd3-9b06-06df5ce7e331


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:51:53,924] Trial 0 finished with value: 0.25869440306656033 and parameters: {'n_estimators': 1400, 'learning_rate': 0.07140827440022966, 'max_depth': 10, 'gamma': 7.28324331733616e-07, 'reg_alpha': 1.3580293329656646e-06, 'reg_lambda': 0.5156060913355952, 'colsample_bytree': 0.9967012173939188, 'subsample': 0.976345994641937}. Best is trial 0 with value: 0.25869440306656033.
[I 2025-10-12 06:51:59,253] Trial 1 finished with value: 0.28291889258571556 and parameters: {'n_estimators': 700, 'learning_rate': 0.006560510499272359, 'max_depth': 6, 'gamma': 3.5391745968127207e-07, 'reg_alpha': 0.00023646953131445172, 'reg_lambda': 1.2760001938088397e-05, 'colsample_bytree': 0.8155612044283131, 'subsample': 0.9465504786035044}. Best is trial 1 with value: 0.28291889258571556.
[I 2025-10-12 06:52:30,282] Trial 2 finished with value: 0.2816864809217181 and parameters: {'n_estimators': 900, 'learning_rate': 0.0026831715877091112, 'max_depth': 11, 'gamma': 2.186785683563009e-05, '


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1800
       learning_rate: 0.01869609535065952
           max_depth: 6
               gamma: 6.330362990851294e-07
           reg_alpha: 1.4263169313172173e-06
          reg_lambda: 0.00040035908394255276
    colsample_bytree: 0.6038326911711578
           subsample: 0.765626525275237

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2563   0.7399    0.1494
Optuna Tuned  0.3107   0.7720    0.1152

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost/SentenceBERT_amazon_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [5]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost', "BERT_amazon_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:57:05,439] A new study created in memory with name: no-name-ff5d6365-e3f9-4750-aee0-91109fa50abd


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:57:10,279] Trial 0 finished with value: 0.2936270449939925 and parameters: {'n_estimators': 900, 'learning_rate': 0.01022878978002057, 'max_depth': 3, 'gamma': 3.851765536071268e-06, 'reg_alpha': 6.137416346215714, 'reg_lambda': 1.0148885954671247e-07, 'colsample_bytree': 0.762908298394016, 'subsample': 0.7034860574806191}. Best is trial 0 with value: 0.2936270449939925.
[I 2025-10-12 06:57:19,464] Trial 1 finished with value: 0.2879160477166301 and parameters: {'n_estimators': 500, 'learning_rate': 0.0022355140900543642, 'max_depth': 9, 'gamma': 0.5937959900289096, 'reg_alpha': 3.967183652218777, 'reg_lambda': 1.0956741444152906e-08, 'colsample_bytree': 0.6198126441475342, 'subsample': 0.8970890663946256}. Best is trial 0 with value: 0.2936270449939925.
[I 2025-10-12 06:57:21,474] Trial 2 finished with value: 0.27788082967072636 and parameters: {'n_estimators': 100, 'learning_rate': 0.23737682050878706, 'max_depth': 3, 'gamma': 1.2247813984306005e-08, 'reg_alpha': 0.0


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1400
       learning_rate: 0.008068835575050684
           max_depth: 5
               gamma: 6.772914451680069e-06
           reg_alpha: 0.0008624392201732306
          reg_lambda: 2.9619917047099495e-05
    colsample_bytree: 0.6152981134204643
           subsample: 0.9115623274277996

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2673   0.7434    0.1664
Optuna Tuned  0.3284   0.7790    0.1404

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/XGBoost/BERT_amazon_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


# Audible

### T5

In [3]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_T5.npy'

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost', "T5_audible_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 07:49:55,590] A new study created in memory with name: no-name-f9b3b67e-cd63-4f70-99be-860e8899f72a


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 07:49:58,744] Trial 0 finished with value: 0.15178164605533756 and parameters: {'n_estimators': 2000, 'learning_rate': 0.05969832654384225, 'max_depth': 8, 'gamma': 0.000254424013142272, 'reg_alpha': 1.235500050968628e-05, 'reg_lambda': 2.2644791957004828e-08, 'colsample_bytree': 0.9397181081577395, 'subsample': 0.6793859949024865}. Best is trial 0 with value: 0.15178164605533756.
[I 2025-10-12 07:50:01,616] Trial 1 finished with value: 0.15755858927078745 and parameters: {'n_estimators': 200, 'learning_rate': 0.009123582115631615, 'max_depth': 6, 'gamma': 0.0003644301167932592, 'reg_alpha': 1.3054767666070627e-08, 'reg_lambda': 1.1156922846704285e-08, 'colsample_bytree': 0.7086739877213518, 'subsample': 0.6464469989583271}. Best is trial 1 with value: 0.15755858927078745.
[I 2025-10-12 07:50:06,265] Trial 2 finished with value: 0.15652383234601497 and parameters: {'n_estimators': 400, 'learning_rate': 0.002313579563248304, 'max_depth': 7, 'gamma': 0.0022384770385062234, 


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1100
       learning_rate: 0.0018968965870062315
           max_depth: 10
               gamma: 5.640796070447442e-06
           reg_alpha: 0.22926442227081334
          reg_lambda: 2.4877998102044897e-08
    colsample_bytree: 0.6493480174553767
           subsample: 0.6224601204161151

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1360   0.6471    0.0354
Optuna Tuned  0.1736   0.6842    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost/T5_audible_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### DistilBERT

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost', "DistilBERT_audible_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:03:42,889] A new study created in memory with name: no-name-3d964220-9f16-4264-b74d-52b96b1dee1e


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 08:03:52,810] Trial 0 finished with value: 0.1357246395522121 and parameters: {'n_estimators': 1900, 'learning_rate': 0.005352386475578895, 'max_depth': 12, 'gamma': 0.09831130906387095, 'reg_alpha': 6.752013433555989e-05, 'reg_lambda': 5.172732270585072e-08, 'colsample_bytree': 0.8865567925393802, 'subsample': 0.6250356623944384}. Best is trial 0 with value: 0.1357246395522121.
[I 2025-10-12 08:04:09,790] Trial 1 finished with value: 0.14735212402383963 and parameters: {'n_estimators': 1800, 'learning_rate': 0.0010707813680702974, 'max_depth': 7, 'gamma': 3.031830709146808e-07, 'reg_alpha': 2.972571782153074, 'reg_lambda': 2.4853165257556937, 'colsample_bytree': 0.6770764203715082, 'subsample': 0.6075417530721918}. Best is trial 1 with value: 0.14735212402383963.
[I 2025-10-12 08:04:19,274] Trial 2 finished with value: 0.14343350610038297 and parameters: {'n_estimators': 2000, 'learning_rate': 0.0021987998660863194, 'max_depth': 4, 'gamma': 6.3598161854637084e-06, 'reg_a


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1200
       learning_rate: 0.04964589045646508
           max_depth: 4
               gamma: 0.0002677988538471694
           reg_alpha: 8.861809386492773
          reg_lambda: 1.1602199873801092e-08
    colsample_bytree: 0.6565874250627063
           subsample: 0.6275328629381611

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1254   0.6197    0.0384
Optuna Tuned  0.1580   0.6584    0.0058

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost/DistilBERT_audible_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [5]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost', "RoBERTa_audible_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:07:54,611] A new study created in memory with name: no-name-b375baf8-f511-456f-ad73-bcf6bda5bf2a


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 08:07:57,656] Trial 0 finished with value: 0.11826346086408743 and parameters: {'n_estimators': 1100, 'learning_rate': 0.13641397300084776, 'max_depth': 8, 'gamma': 0.5251972095130337, 'reg_alpha': 2.0210800845733434, 'reg_lambda': 0.016409587403835895, 'colsample_bytree': 0.9548861119504349, 'subsample': 0.7472315397031726}. Best is trial 0 with value: 0.11826346086408743.
[I 2025-10-12 08:08:06,108] Trial 1 finished with value: 0.15026568982380106 and parameters: {'n_estimators': 600, 'learning_rate': 0.006527048313258541, 'max_depth': 8, 'gamma': 0.2890490924194343, 'reg_alpha': 1.5612332711199837e-05, 'reg_lambda': 3.4209870938679403, 'colsample_bytree': 0.7795353214623953, 'subsample': 0.6102158097872932}. Best is trial 1 with value: 0.15026568982380106.
[I 2025-10-12 08:08:12,095] Trial 2 finished with value: 0.1378047086841689 and parameters: {'n_estimators': 1900, 'learning_rate': 0.01452524178237413, 'max_depth': 10, 'gamma': 9.104655648596284e-08, 'reg_alpha': 4


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1700
       learning_rate: 0.041704808918020704
           max_depth: 3
               gamma: 0.00040558236745778114
           reg_alpha: 8.222380677411646
          reg_lambda: 0.0014570029976043227
    colsample_bytree: 0.7189692161896526
           subsample: 0.8949458581302419

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1196   0.6134    0.0304
Optuna Tuned  0.1513   0.6591    0.0087

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost/RoBERTa_audible_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [6]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost', "SentenceBERT_audible_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:13:43,289] A new study created in memory with name: no-name-70bb897c-0bb3-4add-9275-7352c1aa6a8d


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 08:13:45,800] Trial 0 finished with value: 0.09911832020156747 and parameters: {'n_estimators': 400, 'learning_rate': 0.21721394536341895, 'max_depth': 9, 'gamma': 2.924771098835259e-06, 'reg_alpha': 2.6697937728793476e-07, 'reg_lambda': 0.014517746638275486, 'colsample_bytree': 0.6682772452870566, 'subsample': 0.8953254299011891}. Best is trial 0 with value: 0.09911832020156747.
[I 2025-10-12 08:13:50,616] Trial 1 finished with value: 0.13428192291439783 and parameters: {'n_estimators': 1200, 'learning_rate': 0.0014969113384861615, 'max_depth': 3, 'gamma': 0.0029723898801824055, 'reg_alpha': 0.027825112251448743, 'reg_lambda': 1.993145781757423e-07, 'colsample_bytree': 0.7264820457254721, 'subsample': 0.9329040808296764}. Best is trial 1 with value: 0.13428192291439783.
[I 2025-10-12 08:13:55,034] Trial 2 finished with value: 0.11886634855311526 and parameters: {'n_estimators': 1800, 'learning_rate': 0.034148047057501886, 'max_depth': 11, 'gamma': 0.0018469229634404613, 


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 800
       learning_rate: 0.009832361729469032
           max_depth: 5
               gamma: 1.2815613976135242e-08
           reg_alpha: 0.03737891661072621
          reg_lambda: 7.885425261034666
    colsample_bytree: 0.8571035278677617
           subsample: 0.7245324736567039

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1216   0.6225    0.0249
Optuna Tuned  0.1573   0.6727    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost/SentenceBERT_audible_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [7]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost', "BERT_audible_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:19:30,713] A new study created in memory with name: no-name-404f3e0d-c855-495d-81e2-7d631276885e


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 08:19:35,063] Trial 0 finished with value: 0.13741561788547735 and parameters: {'n_estimators': 700, 'learning_rate': 0.0015354942166168576, 'max_depth': 4, 'gamma': 1.046717842345098e-06, 'reg_alpha': 0.49657584286416206, 'reg_lambda': 1.1319460488888557e-08, 'colsample_bytree': 0.6048409757743393, 'subsample': 0.7686736015292382}. Best is trial 0 with value: 0.13741561788547735.
[I 2025-10-12 08:19:42,528] Trial 1 finished with value: 0.1515785873504109 and parameters: {'n_estimators': 900, 'learning_rate': 0.007886188131980014, 'max_depth': 7, 'gamma': 0.022653241252458635, 'reg_alpha': 2.845588810851065, 'reg_lambda': 1.9753024445276663, 'colsample_bytree': 0.711802796728259, 'subsample': 0.657648384334159}. Best is trial 1 with value: 0.1515785873504109.
[I 2025-10-12 08:19:45,862] Trial 2 finished with value: 0.1438504923776884 and parameters: {'n_estimators': 1300, 'learning_rate': 0.03109764540585518, 'max_depth': 6, 'gamma': 9.49734737707902e-06, 'reg_alpha': 1.0


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1900
       learning_rate: 0.00535347179696795
           max_depth: 9
               gamma: 0.026471897833103275
           reg_alpha: 0.008770058489893892
          reg_lambda: 9.320223991303552
    colsample_bytree: 0.6790515278237815
           subsample: 0.6893394873624709

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1217   0.6189    0.0277
Optuna Tuned  0.1586   0.6682    0.0015

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/s2/XGBoost/BERT_audible_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


# Coursera

### T5

In [1]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost', "T5_coursera_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:39:42,555] A new study created in memory with name: no-name-16d55641-4ea5-433b-a542-d5047ac05eec


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 08:39:47,816] Trial 0 finished with value: 0.21948518242578394 and parameters: {'n_estimators': 700, 'learning_rate': 0.010057376039238822, 'max_depth': 4, 'gamma': 0.0010575766883287694, 'reg_alpha': 0.0014122791745601864, 'reg_lambda': 1.5845633471433906e-06, 'colsample_bytree': 0.9389974308090558, 'subsample': 0.8945461483495819}. Best is trial 0 with value: 0.21948518242578394.
[I 2025-10-12 08:39:51,132] Trial 1 finished with value: 0.19272174320736232 and parameters: {'n_estimators': 1300, 'learning_rate': 0.08074920452149957, 'max_depth': 7, 'gamma': 4.6986702103201985e-08, 'reg_alpha': 0.00734114607234041, 'reg_lambda': 0.002238148301255995, 'colsample_bytree': 0.6323046292842185, 'subsample': 0.8811355491079924}. Best is trial 0 with value: 0.21948518242578394.
[I 2025-10-12 08:40:01,061] Trial 2 finished with value: 0.2166431302257856 and parameters: {'n_estimators': 1100, 'learning_rate': 0.001000060755474371, 'max_depth': 6, 'gamma': 1.8788294698835037e-05, 'r


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1700
       learning_rate: 0.004179852411732094
           max_depth: 8
               gamma: 4.0343798139610845e-08
           reg_alpha: 0.5148126458550106
          reg_lambda: 0.018996292455932196
    colsample_bytree: 0.937040591504369
           subsample: 0.703333270765574

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1603   0.7642    0.1048
Optuna Tuned  0.2337   0.8000    0.0598

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost/T5_coursera_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### DistilBERT

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA/model/code/PADA/embedding/coursera_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost', "DistilBERT_coursera_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
🔥 파일 로드 실패: [Errno 2] No such file or directory: '/content/drive/MyDrive/PADA/model/code/PADA/embedding/coursera_DistilBERT.npy'
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:50:54,541] A new study created in memory with name: no-name-4ef25159-cc8f-4747-9119-125524b86623


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 08:51:04,977] Trial 0 finished with value: 0.21964091788168383 and parameters: {'n_estimators': 500, 'learning_rate': 0.00849490059961909, 'max_depth': 10, 'gamma': 0.0015513672382988627, 'reg_alpha': 0.028957902223955303, 'reg_lambda': 1.1548753681185406, 'colsample_bytree': 0.7903449620139886, 'subsample': 0.9930759759417866}. Best is trial 0 with value: 0.21964091788168383.
[I 2025-10-12 08:51:18,414] Trial 1 finished with value: 0.21917036545542412 and parameters: {'n_estimators': 1000, 'learning_rate': 0.0014430271120228324, 'max_depth': 8, 'gamma': 0.008928187141777321, 'reg_alpha': 0.03604120046284453, 'reg_lambda': 8.327682556249776e-06, 'colsample_bytree': 0.8791893485224529, 'subsample': 0.7474223300850016}. Best is trial 0 with value: 0.21964091788168383.
[I 2025-10-12 08:51:25,469] Trial 2 finished with value: 0.2211233565485295 and parameters: {'n_estimators': 2000, 'learning_rate': 0.014966001044943054, 'max_depth': 4, 'gamma': 6.029635066565396e-07, 'reg_al


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1600
       learning_rate: 0.0017316417023200436
           max_depth: 9
               gamma: 8.013416537124657e-06
           reg_alpha: 3.2788809343542e-08
          reg_lambda: 9.61271152610749e-06
    colsample_bytree: 0.923331524903376
           subsample: 0.8148159046136605

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1603   0.7642    0.1048
Optuna Tuned  0.2340   0.7961    0.0465

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost/DistilBERT_coursera_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [3]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA/embedding/coursera_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost', "RoBERTa_coursera_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
🔥 파일 로드 실패: [Errno 2] No such file or directory: '/content/drive/MyDrive/PADA/model/code/PADA/embedding/coursera_RoBERTa.npy'
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 09:03:55,931] A new study created in memory with name: no-name-e6d200ff-aedc-4d67-84d7-cd63de8437eb


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 09:04:01,945] Trial 0 finished with value: 0.2137037510017442 and parameters: {'n_estimators': 500, 'learning_rate': 0.001240984161983602, 'max_depth': 6, 'gamma': 3.5043947333872805e-08, 'reg_alpha': 0.43165766468086125, 'reg_lambda': 0.011084616392445533, 'colsample_bytree': 0.6570404521392766, 'subsample': 0.7780872742000392}. Best is trial 0 with value: 0.2137037510017442.
[I 2025-10-12 09:04:15,527] Trial 1 finished with value: 0.2225956478693133 and parameters: {'n_estimators': 2000, 'learning_rate': 0.004241455897049877, 'max_depth': 10, 'gamma': 0.009575934233715454, 'reg_alpha': 0.014334094599126993, 'reg_lambda': 0.05672147847017883, 'colsample_bytree': 0.6668640709051427, 'subsample': 0.6419534283789933}. Best is trial 1 with value: 0.2225956478693133.
[I 2025-10-12 09:04:18,464] Trial 2 finished with value: 0.20585240510755154 and parameters: {'n_estimators': 200, 'learning_rate': 0.01174580416551097, 'max_depth': 3, 'gamma': 0.8547020186907851, 'reg_alpha': 0


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1900
       learning_rate: 0.004327374600152431
           max_depth: 7
               gamma: 0.002944426441975355
           reg_alpha: 9.763296414691562
          reg_lambda: 8.339035445287745e-08
    colsample_bytree: 0.7594969168921509
           subsample: 0.7398686876187779

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1603   0.7642    0.1048
Optuna Tuned  0.2376   0.7999    0.0646

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost/RoBERTa_coursera_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [1]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost', "SentenceBERT_coursera_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 10:50:17,341] A new study created in memory with name: no-name-95d277af-1730-4bae-a9c3-9f807b745d56


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 10:50:29,150] Trial 0 finished with value: 0.200705686973591 and parameters: {'n_estimators': 500, 'learning_rate': 0.011522551805179963, 'max_depth': 12, 'gamma': 5.349006721285702e-06, 'reg_alpha': 0.0006643874669789749, 'reg_lambda': 1.3309480347013896, 'colsample_bytree': 0.7818435155033598, 'subsample': 0.9698116969629096}. Best is trial 0 with value: 0.200705686973591.
[I 2025-10-12 10:50:32,487] Trial 1 finished with value: 0.18619314849034285 and parameters: {'n_estimators': 400, 'learning_rate': 0.0039042884034475975, 'max_depth': 4, 'gamma': 3.316145881837368e-05, 'reg_alpha': 5.980251158537709e-07, 'reg_lambda': 1.640758170955621, 'colsample_bytree': 0.6897987526591997, 'subsample': 0.9341749555823431}. Best is trial 0 with value: 0.200705686973591.
[I 2025-10-12 10:50:35,382] Trial 2 finished with value: 0.1894850299320619 and parameters: {'n_estimators': 2000, 'learning_rate': 0.07553052616035043, 'max_depth': 4, 'gamma': 0.001125015960079503, 'reg_alpha': 0.


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1600
       learning_rate: 0.0012813238944652562
           max_depth: 12
               gamma: 0.00011232542335100464
           reg_alpha: 7.054097407776223e-06
          reg_lambda: 0.009377009126500843
    colsample_bytree: 0.6730213285935831
           subsample: 0.7080825813913985

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1692   0.7478    0.1026
Optuna Tuned  0.2121   0.7883    0.0037

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost/SentenceBERT_coursera_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [2]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost', "BERT_coursera_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:09:27,896] A new study created in memory with name: no-name-2b1fee27-30ad-48d2-a736-92f3b1500acf


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 11:09:34,077] Trial 0 finished with value: 0.17541929845436044 and parameters: {'n_estimators': 800, 'learning_rate': 0.06990911692909418, 'max_depth': 12, 'gamma': 3.716258451761101e-08, 'reg_alpha': 1.7393954948463752, 'reg_lambda': 0.1289888067460368, 'colsample_bytree': 0.9101351535824339, 'subsample': 0.9176244338154769}. Best is trial 0 with value: 0.17541929845436044.
[I 2025-10-12 11:09:39,062] Trial 1 finished with value: 0.20092264153501738 and parameters: {'n_estimators': 400, 'learning_rate': 0.01016623618875081, 'max_depth': 6, 'gamma': 0.0013600807235764694, 'reg_alpha': 0.05185742056627907, 'reg_lambda': 0.5575855947608294, 'colsample_bytree': 0.6756473229841975, 'subsample': 0.9293601920066777}. Best is trial 1 with value: 0.20092264153501738.
[I 2025-10-12 11:09:55,372] Trial 2 finished with value: 0.20094416795259099 and parameters: {'n_estimators': 900, 'learning_rate': 0.0024842391262942885, 'max_depth': 9, 'gamma': 1.2238972916474687e-08, 'reg_alpha':


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1300
       learning_rate: 0.012589245667875828
           max_depth: 4
               gamma: 3.1463744519804986e-05
           reg_alpha: 5.236867746311568e-06
          reg_lambda: 1.8385314882970543
    colsample_bytree: 0.9609850175095549
           subsample: 0.7384491336472978

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1700   0.7556    0.1132
Optuna Tuned  0.2164   0.7894    0.0765

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/XGBoost/BERT_coursera_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


# Hotel

### T5

In [5]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost', "T5_hotel_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:21:39,422] A new study created in memory with name: no-name-8aa667fe-d84c-4cc7-97e9-427bb6194c7b


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 11:21:42,588] Trial 0 finished with value: 0.1024720115828601 and parameters: {'n_estimators': 600, 'learning_rate': 0.17543863074167704, 'max_depth': 10, 'gamma': 5.106345358242599e-07, 'reg_alpha': 0.017222578693367116, 'reg_lambda': 1.3853692003111273, 'colsample_bytree': 0.8271851823905481, 'subsample': 0.676388771412704}. Best is trial 0 with value: 0.1024720115828601.
[I 2025-10-12 11:21:56,369] Trial 1 finished with value: 0.15246437908348814 and parameters: {'n_estimators': 1200, 'learning_rate': 0.003806426894904188, 'max_depth': 11, 'gamma': 1.0673733107507555e-06, 'reg_alpha': 1.6141209976540235e-07, 'reg_lambda': 0.002431482022533605, 'colsample_bytree': 0.8071744093479981, 'subsample': 0.9701142596804327}. Best is trial 1 with value: 0.15246437908348814.
[I 2025-10-12 11:22:01,171] Trial 2 finished with value: 0.1521374879145544 and parameters: {'n_estimators': 300, 'learning_rate': 0.0059440532537950565, 'max_depth': 7, 'gamma': 0.00038205362763683094, 'reg_


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1900
       learning_rate: 0.0017147297322607925
           max_depth: 6
               gamma: 7.399266228735294e-05
           reg_alpha: 1.0408674760353577e-08
          reg_lambda: 1.0887649965600925e-07
    colsample_bytree: 0.7555097767585653
           subsample: 0.7070085727114317

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1278   0.5931    0.0176
Optuna Tuned  0.1645   0.6330    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost/T5_hotel_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### DistilBERT

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost', "DistilBERT_hotel_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:28:49,523] A new study created in memory with name: no-name-f00df441-59a8-498a-967d-95a832f3fd93


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 11:28:51,842] Trial 0 finished with value: 0.13419031877489637 and parameters: {'n_estimators': 600, 'learning_rate': 0.16091394485573354, 'max_depth': 7, 'gamma': 0.6480588076073167, 'reg_alpha': 9.121421009190561e-07, 'reg_lambda': 2.575716347035618e-07, 'colsample_bytree': 0.7910519700856177, 'subsample': 0.9323385177499363}. Best is trial 0 with value: 0.13419031877489637.
[I 2025-10-12 11:29:00,153] Trial 1 finished with value: 0.1541513627107805 and parameters: {'n_estimators': 1100, 'learning_rate': 0.0015347497405630121, 'max_depth': 6, 'gamma': 0.853441199567755, 'reg_alpha': 0.001027645919769887, 'reg_lambda': 0.0028031197744448986, 'colsample_bytree': 0.9989589471840241, 'subsample': 0.6157698847269195}. Best is trial 1 with value: 0.1541513627107805.
[I 2025-10-12 11:29:17,254] Trial 2 finished with value: 0.15346630784645893 and parameters: {'n_estimators': 1600, 'learning_rate': 0.0012988846935399856, 'max_depth': 8, 'gamma': 0.0016718353783205463, 'reg_alph


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1500
       learning_rate: 0.007471897543492743
           max_depth: 5
               gamma: 3.179420883499346e-08
           reg_alpha: 3.694489228818821e-08
          reg_lambda: 0.8388755255220983
    colsample_bytree: 0.6592027754151295
           subsample: 0.6209294393349328

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1297   0.5909    0.0244
Optuna Tuned  0.1601   0.6290    0.0012

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost/DistilBERT_hotel_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [7]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost', "RoBERTa_hotel_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:33:39,676] A new study created in memory with name: no-name-faea3d75-1ccc-47d5-9559-8d3fff960d5b


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 11:33:42,456] Trial 0 finished with value: 0.1479778898051613 and parameters: {'n_estimators': 400, 'learning_rate': 0.006004898048093438, 'max_depth': 3, 'gamma': 0.0001856120169547232, 'reg_alpha': 2.562292586251243e-08, 'reg_lambda': 1.924455666665389e-08, 'colsample_bytree': 0.7224458628762738, 'subsample': 0.7895033971174072}. Best is trial 0 with value: 0.1479778898051613.
[I 2025-10-12 11:33:47,419] Trial 1 finished with value: 0.14426372280966052 and parameters: {'n_estimators': 100, 'learning_rate': 0.0015096805211048919, 'max_depth': 12, 'gamma': 4.743858709113505e-07, 'reg_alpha': 5.530499734809304, 'reg_lambda': 0.0002245412664201858, 'colsample_bytree': 0.7197704939999853, 'subsample': 0.7511400333191014}. Best is trial 0 with value: 0.1479778898051613.
[I 2025-10-12 11:33:50,028] Trial 2 finished with value: 0.14970562208185206 and parameters: {'n_estimators': 1000, 'learning_rate': 0.02826318183827815, 'max_depth': 4, 'gamma': 2.871447164108296e-06, 'reg_al


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1500
       learning_rate: 0.004282006686829183
           max_depth: 6
               gamma: 2.9116415465995856e-05
           reg_alpha: 0.1651081279728344
          reg_lambda: 3.1174097408692453e-07
    colsample_bytree: 0.704694088192261
           subsample: 0.8060251453106023

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1210   0.5840    0.0163
Optuna Tuned  0.1562   0.6256    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost/RoBERTa_hotel_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [8]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost', "SentenceBERT_hotel_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:39:45,194] A new study created in memory with name: no-name-2bed6ec7-4ca8-44ab-b24f-d2f41a27330b


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 11:39:53,271] Trial 0 finished with value: 0.14463507593216085 and parameters: {'n_estimators': 1700, 'learning_rate': 0.006587721260851636, 'max_depth': 11, 'gamma': 8.273247762122286e-08, 'reg_alpha': 1.9061343902298392e-05, 'reg_lambda': 0.002119149751561805, 'colsample_bytree': 0.764049767370023, 'subsample': 0.7893101296884232}. Best is trial 0 with value: 0.14463507593216085.
[I 2025-10-12 11:39:55,428] Trial 1 finished with value: 0.13599538872119185 and parameters: {'n_estimators': 400, 'learning_rate': 0.19643360919678785, 'max_depth': 5, 'gamma': 0.009568011610142408, 'reg_alpha': 3.1798461971408845, 'reg_lambda': 0.004280516296619961, 'colsample_bytree': 0.9386509617116922, 'subsample': 0.890362753135882}. Best is trial 0 with value: 0.14463507593216085.
[I 2025-10-12 11:39:58,358] Trial 2 finished with value: 0.11964309830792566 and parameters: {'n_estimators': 1800, 'learning_rate': 0.18270713584506798, 'max_depth': 9, 'gamma': 2.2086911211556925e-05, 'reg_al


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 700
       learning_rate: 0.008144712035439313
           max_depth: 5
               gamma: 9.379399742777656e-06
           reg_alpha: 0.00014370851358462986
          reg_lambda: 0.7687859640550706
    colsample_bytree: 0.8175503429242595
           subsample: 0.7038043257837097

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1224   0.5797    0.0173
Optuna Tuned  0.1551   0.6271    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost/SentenceBERT_hotel_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [9]:
# === 1. 라이브러리 임포트 ===
! pip install optuna xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
from xgboost.callback import EarlyStopping

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: XGBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'booster': 'gbtree',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Regularization (lambda_l1)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Regularization (lambda_l2)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: XGBClassifier 사용 및 GPU 설정 (device='cuda')
    model = xgb.XGBClassifier(device='cuda',early_stopping_rounds = 100, **params)

    # 🌟 변경점: XGBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              verbose=False)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    baseline_model = xgb.XGBClassifier(device='cuda', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> xgb.XGBClassifier, device='cuda'
    final_model = xgb.XGBClassifier(device='cuda', objective='binary:logistic', verbosity=0, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost', "BERT_hotel_with_s2_predictions_XGBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}'_XGBoost파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:44:04,157] A new study created in memory with name: no-name-7123f96d-b6f7-4def-b0cf-4038390ac652


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 11:44:10,321] Trial 0 finished with value: 0.14558009225205726 and parameters: {'n_estimators': 1700, 'learning_rate': 0.017413055812059423, 'max_depth': 11, 'gamma': 5.679962825596535e-06, 'reg_alpha': 0.00030602637291877956, 'reg_lambda': 1.910306154170625e-07, 'colsample_bytree': 0.8601527010858905, 'subsample': 0.9156320185706885}. Best is trial 0 with value: 0.14558009225205726.
[I 2025-10-12 11:44:13,230] Trial 1 finished with value: 0.1552397508461652 and parameters: {'n_estimators': 300, 'learning_rate': 0.007866516904641708, 'max_depth': 5, 'gamma': 0.0004356793207768946, 'reg_alpha': 0.2689749916511799, 'reg_lambda': 6.026396177100992e-08, 'colsample_bytree': 0.7330357253632033, 'subsample': 0.8728794916438163}. Best is trial 1 with value: 0.1552397508461652.
[I 2025-10-12 11:44:29,842] Trial 2 finished with value: 0.15296970850120484 and parameters: {'n_estimators': 600, 'learning_rate': 0.005072860587761919, 'max_depth': 12, 'gamma': 0.00011558118060139505, 'r


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1900
       learning_rate: 0.004809149811833711
           max_depth: 5
               gamma: 6.667852095841122e-08
           reg_alpha: 1.965947659609614e-05
          reg_lambda: 0.004260966896567145
    colsample_bytree: 0.8060965564709778
           subsample: 0.9534044492143692

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1317   0.5919    0.0210
Optuna Tuned  0.1599   0.6290    0.0024

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/XGBoost/BERT_hotel_with_s2_predictions_XGBoost.csv'_XGBoost파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!
